In [1]:
from helpers.data_preparation import * 
from helpers.nlp_functions import *
from helpers.time_series_functions import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [ ]:
stock_list = stockListFromURL(market="london")
stock_data = stockDataFromYf(stock_list)
# stock_list = filterStocksByVolume(stock_data, largest_n=200)
price_data = stock_data["Close"][stock_list]

threshold = 0.3 * len(price_data)
price_data = price_data.dropna(thresh=len(price_data) - threshold, axis=1)
price_data = price_data.ffill()
price_data = price_data.bfill()

stock_list = list(price_data.columns)
stock2id = stocks2id(stock_list)
stock_data = stockDataFromYf(stock_list)

textual_data = fetchTextualInformation(stock_list, stock2id)

In [59]:
pd.DataFrame(textual_data).T.sample(5)

,ticker_id,code,name,sector,industry,description
SAIN.L,197,SAIN.L,SCOTTISH AMERICAN INVESTMENT CO,Financial Services,Asset Management,The Scottish American Investment Company P.L.C...
BCG.L,21,BCG.L,BALTIC CLASSIFIEDS GROUP PLC OR,Communication Services,Internet Content & Information,Baltic Classifieds Group PLC owns and operates...
SPT.L,208,SPT.L,SPIRENT COMMUNICATIONS PLC ORD,Technology,Software - Infrastructure,Spirent Communications plc provides automated ...
PPH.L,179,PPH.L,PPHE HOTEL GROUP LIMITED ORD NP,Consumer Cyclical,Lodging,"PPHE Hotel Group Limited owns, co-owns, develo..."
PRTC.L,182,PRTC.L,PURETECH HEALTH PLC ORD 1P,Healthcare,Biotechnology,"PureTech Health plc, engages in the developmen..."


In [60]:
stock_list = textual_data.keys()

In [61]:
len(stock_list)

237

In [62]:
tokenizer, nlp_model = load_model()
description_feature_vectors = {}

for i, key in enumerate(list(textual_data.keys())):
    desc = textual_data[key]["description"]
    feature_vector = extract_features(text=desc, tokenizer=tokenizer, model=nlp_model)
    description_feature_vectors[i] = feature_vector

title = "london"
with open(f'cold_data/description_feature_vectors{title}.pkl', 'wb') as f:
    pickle.dump(description_feature_vectors, f)

price_data = price_data[list(textual_data.keys())]
sim_metrics = ["pearson", "dtw", "dist_corr", "euclidean"]
for metric in sim_metrics:
    ts_sim = calculate_historical_ts_similarities(price_data, start_date="2023-01-01", end_date="2023-06-01", 
                                                  similarity_metric=metric, path=f"range_similarities_{title}.pkl")

    with open(f'calculated_data/historical_ts_sim_london_{metric}.pkl', 'wb') as f:
        pickle.dump(ts_sim, f)

100%|██████████| 237/237 [01:58<00:00,  2.01it/s]


In [ ]:
price_data["Date"] = stock_data.index

price_data.to_csv(f"cold_data/stock_data{title}.csv", index=False)
pd.DataFrame(textual_data).to_csv(f"cold_data/textual_information{title}.csv", index=False)

In [80]:
s = pd.DataFrame(columns=list(price_data.drop("Date", axis=1).columns))
s.loc[0] = (list(stocks2id(stock_list).values()))
s.to_csv(f"cold_data/stock2id{title}.csv", index=False)

In [81]:
s

,3IN.L,FOUR.L,ABDN.L,APEO.L,ATST.L,ATT.L,AO.L,APAX.L,ASCL.L,ASHM.L,...,WPS.L,WOSG.L,JDW.L,SMWH.L,WTAN.L,WIZZ.L,WG.L,WKP.L,WWH.L,ZIG.L
0,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
